<a href="https://colab.research.google.com/github/aguessous/projetsmlops-3/blob/main/Taxi_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Setup**

### **Install libraries**

In [1]:
# Gymnasium : bibli. d’environnements RL (inclut Taxi‑v3)
# pygame/pyglet : rendu graphique
# numpy : calcul matriciel
# tqdm : jolies barres de progression
# huggingface_hub : pour envoyer le modèle sur le Hub
# pickle5 : sérialisation d’objets Python
!pip install gymnasium pygame numpy huggingface_hub pyyaml==6.0 imageio imageio_ffmpeg pyglet==1.5.1 tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.1 MB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2


In [2]:
# ---------------------------------------------------------------------
# DEPENDANCES SYSTÈME (rendu vidéo sous Colab)
# ---------------------------------------------------------------------
!sudo apt-get update
!sudo apt-get install -y python3-opengl        # OpenGL pour le rendu
!apt install ffmpeg xvfb                       # ffmpeg = encodage vidéo
!pip3 install pyvirtualdisplay                 # écran virtuel (pas d’affichage physique)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,702 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,735 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,951 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpad

### **Restart Runtime**

In [ ]:
# REDÉMARRAGE DU RUNTIME (Colab nécessite un restart après apt-get)
import os
os.kill(os.getpid(), 9) # tue le process => Colab redémarre

### **Importing libraries**

In [20]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm
import pickle

from tqdm.notebook import tqdm

### **Startup Virtual Display**

In [2]:
#  AFFICHAGE VIRTUEL (indispensable sous Colab pour voir des vidéos)
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(3840, 2160))
virtual_display.start()

# **2. Environment**

### **Initialize Environment**

In [4]:
# INITIALISATION DE L’ENVIRONNEMENT TAXI‑V3
env = gym.make("Taxi-v3", render_mode="rgb_array")

Passenger locations:

0: Red

1: Green

2: Yellow

3: Blue

4: In taxi

Destinations:

0: Red

1: Green

2: Yellow

3: Blue

In [5]:
# ----- dimensions de l’espace d’états & d’actions --------------------
state_space = env.observation_space.n      # 500 états discrets
print(f"There are {state_space} possible states")

There are 500 possible states


0: Move south (down)

1: Move north (up)

2: Move east (right)

3: Move west (left)

4: Pickup passenger

5: Drop off passenger

In [6]:
action_space = env.action_space.n #   6 actions possibles
print("There are ", action_space, " possible actions")

There are  6  possible actions


### **Initialize Q Table**

In [7]:
# CREATION DE LA Q‑TABLE (matrice [états × actions])
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

Qtable_taxi = initialize_q_table(state_space, action_space)
print(Qtable_taxi)
print("Q-table shape: ", Qtable_taxi.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Q-table shape:  (500, 6)


# **3. Training & Evaluating**

Define Policies for training

### **greedy policy**

In [8]:
def greedy_policy(Qtable, state):
  # Exploitation pure : on prend l’action dont la valeur Q est maximale
  action = np.argmax(Qtable[state][:])

  return action

### **epsilon greedy policy**

In [9]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  random_num = random.uniform(0,1)
  # if random_num > greater than epsilon --> exploitation
  if random_num > epsilon:
    # Prendre action avec Q est maximale
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()

  return action

hyper parameters

In [10]:
# ---------------------- TRAINING -------------------------------------
n_training_episodes = 1500   # nombre total d’épisodes d’apprentissage
learning_rate       = 0.81   # α  (0<α≤1)  → “poids” donné à la nouvelle info
                             # α élevé = on apprend vite mais on oublie vite

# ---------------------- ENVIRONNEMENT --------------------------------
env_id = "Taxi-v3"           # Name of the environment
max_steps = 100              # limite d’actions par épisode (évite les boucles)
gamma     = 0.95             # γ  (0<γ≤1) pondère l’importance du futur
                             # γ≈0 : vision court‑terme / γ≈1 : long‑terme

# ---------------------- EXPLORATION ----------------------------------
max_epsilon = 1.0   # ε_initial : 100 % d’exploration au tout début
min_epsilon = 0.05  # ε_final   : on garde 5 % de hasard même en fin
decay_rate  = 0.005 # vitesse à laquelle ε décroît → courbe exp

# ---------------------- EVALUATION -----------------------------------
n_eval_episodes = 100  # nombre d’épisodes de test (ε=0)
#  graine fixe : liste de nombres utilisés pour env.reset(seed=…)
#  ≈ « plan de départ imposé » pour comparer tous les agents
#  sur EXACTEMENT les mêmes situations initiales.
eval_seed = [16,54,165,177,191,191,120,80,149,178,48,38,6,125,174,73,50,172,100,148,146,6,25,40,68,148,49,167,9,97,164,176,61,7,54,55,
 161,131,184,51,170,12,120,113,95,126,51,98,36,135,54,82,45,95,89,59,95,124,9,113,58,85,51,134,121,169,105,21,30,11,50,65,12,43,82,145,152,97,106,55,31,85,38,
 112,102,168,123,97,21,83,158,26,80,63,5,81,32,11,28,148]



Training function

In [11]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Reset the environment
    state, info = env.reset()
    step = 0
    terminated = False
    truncated = False

    # repeat
    for step in range(max_steps):
      # Choose the action At using epsilon greedy policy
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # Take action At and observe Rt+1 and St+1
      # Take the action (a) and observe the outcome state(s') and reward (r)
      new_state, reward, terminated, truncated, info = env.step(action)

      # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      # If terminated or truncated finish the episode
      if terminated or truncated:
        break

      # Our next state is the new state
      state = new_state
  return Qtable

Train

In [12]:
# ---------- Lancement de l’entraînement ----------
Qtable_taxi = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_taxi)
Qtable_taxi

  0%|          | 0/1500 [00:00<?, ?it/s]

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  2.74491037,   2.20192272,   1.043355  ,   2.50013545,
          5.20997639,  -5.48422281],
       [  5.61029013,   1.42903062,   7.49061081,   8.78937782,
         10.9512375 ,   0.34534891],
       ...,
       [  8.28145227,  12.58025   ,   8.29130011,   7.11387455,
         -4.06534213,   1.58838783],
       [ -4.46276061,  -3.95499011,  -4.49850929,  -4.29869635,
         -8.1       , -13.26177643],
       [ -0.9639    ,  -1.587195  ,  10.25247765,  17.99553806,
         -8.1       ,  -8.1       ]])

model card

In [14]:
#  SAUVEGARDE DES MÉTADONNÉES DU MODÈLE
model = {
    "env_id": env_id,
    "max_steps": max_steps,
    "n_training_episodes": n_training_episodes,
    "n_eval_episodes": n_eval_episodes,
    "eval_seed": eval_seed, # <- graine fixe pour évaluer

    "learning_rate": learning_rate,
    "gamma": gamma,

    "max_epsilon": max_epsilon,
    "min_epsilon": min_epsilon,
    "decay_rate": decay_rate,

    "qtable": Qtable_taxi
}

## Record Video

record video

In [15]:
# ENREGISTRER UNE VIDÉO DE L’AGENT
def record_video(env, Qtable, out_directory, fps=1, max_steps=20):
    """
    Exécute un épisode (en exploitation pure) et enregistre chaque frame.
    fps = 1 car le jeu avance d’une action par seconde dans la vidéo.
    """
    images = []
    terminated = False
    truncated = False
    state, info = env.reset(seed=random.randint(0,500))
    img = env.render()
    images.append(img)
    step_count = 0
    while (not terminated or not truncated) and step_count < max_steps:
        action = np.argmax(Qtable[state][:])
        state, reward, terminated, truncated, info = env.step(action)
        img = env.render()
        images.append(img)
        step_count += 1

    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

record single video

In [16]:
video_name = "replay.mp4"
record_video(env, model["qtable"], video_name, 1)

display video in notebook

In [17]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# **4. Agent Evaluation**

In [18]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
    Mesure le score moyen sur n_eval_episodes sans exploration (ε=0).
    La graine fixe garantit que tous les étudiants comparent
    les agents sur les mêmes positions de départ.
    """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state, info = env.reset(seed=seed[episode])
    else:
      state, info = env.reset()
    step = 0
    truncated = False
    terminated = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Take the action (index) that have the maximum expected future reward given that state
      action = greedy_policy(Q, state)
      new_state, reward, terminated, truncated, info = env.step(action)
      total_rewards_ep += reward

      if terminated or truncated:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

In [19]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_taxi, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Mean_reward=7.36 +/- 2.85


# **5. Push to Hub**

In [21]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json

In [22]:
env_id = "Taxi-v3"  # identifiant stocké (pour Hugging Face Hub)
def push_to_hub(
    repo_id, model, env, video_fps=1, local_repo_path="hub"
):
    """
    Pipeline complet :
      1. crée (ou récupère) le dépôt sur Hugging Face
      2. sauvegarde le modèle (pickle)
      3. évalue l’agent et stocke les metrics
      4. crée un README + métadonnées
      5. enregistre une vidéo .mp4
      6. charge le tout sur le Hub
    Les étudiants découvrent ainsi le workflow MLOps “basique”.
    """
    _, repo_name = repo_id.split("/")

    eval_env = env
    api = HfApi()

    # 1) crée le dépôt (exist_ok=True => pas d’erreur s’il existe déjà)
    repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
    )

     # 2) clone dans /hub (snapshot_download = cache local)
    repo_local_path = Path(snapshot_download(repo_id=repo_id))

    # 3) sérialise la Q‑table
    if env.spec.kwargs.get("map_name"):
        model["map_name"] = env.spec.kwargs.get("map_name")
        if env.spec.kwargs.get("is_slippery", "") == False:
            model["slippery"] = False

    # Pickle the model
    with open((repo_local_path) / "q-learning.pkl", "wb") as f:
        pickle.dump(model, f)

    # Step 4: Evaluate the model and build JSON with evaluation metrics
    mean_reward, std_reward = evaluate_agent(
        eval_env, model["max_steps"], model["n_eval_episodes"], model["qtable"], model["eval_seed"]
    )

    evaluate_data = {
        "env_id": model["env_id"],
        "mean_reward": mean_reward,
        "n_eval_episodes": model["n_eval_episodes"],
        "eval_datetime": datetime.datetime.now().isoformat()
    }

    # Write a JSON file called "results.json" that will contain the
    # evaluation results
    with open(repo_local_path / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # Step 5: Create the model card
    env_name = model["env_id"]
    if env.spec.kwargs.get("map_name"):
        env_name += "-" + env.spec.kwargs.get("map_name")

    if env.spec.kwargs.get("is_slippery", "") == False:
        env_name += "-" + "no_slippery"

    metadata = {}
    metadata["tags"] = [env_name, "q-learning", "reinforcement-learning", "custom-implementation"]

    # Add metrics
    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
    )

    # Merges both dictionaries
    metadata = {**metadata, **eval}

    model_card = f"""
  # **Q-Learning** Agent playing1 **{env_id}**
  This is a trained model of a **Q-Learning** agent playing **{env_id}** .

  ## Usage

  ```python

  model = load_from_hub(repo_id="{repo_id}", filename="q-learning.pkl")

  # Don't forget to check if you need to add additional attributes (is_slippery=False etc)
  env = gym.make(model["env_id"])
  ```
  """

    evaluate_agent(env, model["max_steps"], model["n_eval_episodes"], model["qtable"], model["eval_seed"])

    readme_path = repo_local_path / "README.md"
    readme = ""
    print(readme_path.exists())
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
            readme = f.read()
    else:
        readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
        f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)

    # Step 6: Record a video
    video_path = repo_local_path / "replay.mp4"
    record_video(env, model["qtable"], video_path, video_fps)

    # Step 7. Push everything to the Hub
    api.upload_folder(
        repo_id=repo_id,
        folder_path=repo_local_path,
        path_in_repo=".",
    )

    print("Your model is pushed to the Hub. You can view your model here: ", repo_url)

In [23]:
from huggingface_hub import notebook_login
notebook_login()

In [24]:
username = "aguessous" # FILL THIS
repo_name = "Taxi-v3" # FILL THIS
push_to_hub(
    repo_id=f"{username}/{repo_name}",
    model=model,
    env=env)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

False


q-learning.pkl:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

replay.mp4:   0%|          | 0.00/130k [00:00<?, ?B/s]

Your model is pushed to the Hub. You can view your model here:  https://huggingface.co/aguessous/Taxi-v3


# **Load from hub***

In [25]:
from urllib.error import HTTPError

from huggingface_hub import hf_hub_download


def load_from_hub(repo_id: str, filename: str) -> str:
    """
    Download a model from Hugging Face Hub.
    :param repo_id: id of the model repository from the Hugging Face Hub
    :param filename: name of the model zip file from the repository
    """
    # Get the model from the Hub, download and cache the model on your local disk
    pickle_model = hf_hub_download(
        repo_id=repo_id,
        filename=filename
    )

    with open(pickle_model, 'rb') as f:
      downloaded_model_file = pickle.load(f)

    return downloaded_model_file

In [26]:
model = load_from_hub(repo_id="aguessous/Taxi-v3", filename="q-learning.pkl") # Try to use another model

print(model)
env = gym.make(model["env_id"])

evaluate_agent(env, model["max_steps"], model["n_eval_episodes"], model["qtable"], model["eval_seed"])

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


q-learning.pkl:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

{'env_id': 'Taxi-v3', 'max_steps': 100, 'n_training_episodes': 1500, 'n_eval_episodes': 100, 'eval_seed': [16, 54, 165, 177, 191, 191, 120, 80, 149, 178, 48, 38, 6, 125, 174, 73, 50, 172, 100, 148, 146, 6, 25, 40, 68, 148, 49, 167, 9, 97, 164, 176, 61, 7, 54, 55, 161, 131, 184, 51, 170, 12, 120, 113, 95, 126, 51, 98, 36, 135, 54, 82, 45, 95, 89, 59, 95, 124, 9, 113, 58, 85, 51, 134, 121, 169, 105, 21, 30, 11, 50, 65, 12, 43, 82, 145, 152, 97, 106, 55, 31, 85, 38, 112, 102, 168, 123, 97, 21, 83, 158, 26, 80, 63, 5, 81, 32, 11, 28, 148], 'learning_rate': 0.81, 'gamma': 0.95, 'max_epsilon': 1.0, 'min_epsilon': 0.05, 'decay_rate': 0.005, 'qtable': array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  2.74491037,   2.20192272,   1.043355  ,   2.50013545,
          5.20997639,  -5.48422281],
       [  5.61029013,   1.42903062,   7.49061081,   8.78937782,
         10.9512375 ,   0.34534891],
       ...,
       [  8.28145227,  12.58025  

  0%|          | 0/100 [00:00<?, ?it/s]

(np.float64(7.36), np.float64(2.8478764018124103))